In [53]:
import string
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import re
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from urllib.parse import urlparse
from googlesearch import search
!pip install tld
from tld import get_tld
import os.path

# Texto

In [ ]:
df=pd.read_csv('phishing_nophishing.csv')
df.head()

In [55]:
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ' '))

In [56]:
import nltk
nltk.download('stopwords')
stemmer = PorterStemmer()
corpus = []
stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
    text = df['text'].iloc[i].lower()
    text = text.translate(str.maketrans('','', string.punctuation)).split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
def vec_text(text, vectorizer):
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')

    stemmer = PorterStemmer()
    stopwords_set = set(stopwords.words('english'))

    text = text.lower()
    text = text.translate(str.maketrans('','', string.punctuation)).split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)

    vec = vectorizer.transform([text])

    return vec

## Model Text

In [58]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus).toarray()
y = df.label_num

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [59]:
clf_text = RandomForestClassifier(n_jobs=-1)

clf_text.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [60]:
clf_text.score(X_test, y_test)

0.9797101449275363

In [61]:
import joblib

# Guardar el modelo de texto
joblib.dump(clf_text, 'modelo_phishing_texto.pkl')
print("Modelo basado en texto guardado exitosamente.")


Modelo basado en texto guardado exitosamente.


# Enlaces

In [62]:
df=pd.read_csv('urlset.csv', encoding='ISO-8859-1')
df.head()

,domain,label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1
3,mail.printakid.com/www.online.americanexpress....,1
4,thewhiskeydregs.com/wp-content/themes/widescre...,1


In [63]:
df.info()
df.label.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96011 entries, 0 to 96010
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   domain  96007 non-null  object
 1   label   95915 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


array(['1', nan, "0.7'8049", '0.770083', '0'], dtype=object)

In [64]:
seleccion = (df.label.isnull()) | (df.label=="0.7'8049") | (df.label=="0.770083")
df = df[~seleccion]

## Preprocesamiento

In [65]:
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

def google_index(url):
    site = search(url, 5)
    return 1 if site else 0

def count_dot(url):
    count_dot = url.count('.')
    return count_dot

def count_www(url):
    url.count('www')
    return url.count('www')

def count_atrate(url):

    return url.count('@')

def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

def count_https(url):
    return url.count('https')

def count_http(url):
    return url.count('http')

def count_per(url):
    return url.count('%')

def count_ques(url):
    return url.count('?')

def count_hyphen(url):
    return url.count('-')

def count_equal(url):
    return url.count('=')

def url_length(url):
    return len(url)

def hostname_length(url):
    return len(urlparse(url).netloc)

def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0

def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1


In [66]:
df['use_of_ip'] = df['domain'].apply(lambda i: having_ip_address(str(i)))
df['abnormal_url'] = df['domain'].apply(lambda i: abnormal_url(str(i)))
df['google_index'] = df['domain'].apply(lambda i: google_index(i))
df['count.'] = df['domain'].apply(lambda i: count_dot(str(i)))
df['count-www'] = df['domain'].apply(lambda i: count_www(str(i)))
df['count@'] = df['domain'].apply(lambda i: count_atrate(str(i)))
df['count_dir'] = df['domain'].apply(lambda i: no_of_dir(str(i)))
df['count_embed_domian'] = df['domain'].apply(lambda i: no_of_embed(str(i)))
df['short_url'] = df['domain'].apply(lambda i: shortening_service(str(i)))
df['count-https'] = df['domain'].apply(lambda i : count_https(str(i)))
df['count-http'] = df['domain'].apply(lambda i : count_http(str(i)))
df['count%'] = df['domain'].apply(lambda i : count_per(str(i)))
df['count?'] = df['domain'].apply(lambda i: count_ques(str(i)))
df['count-'] = df['domain'].apply(lambda i: count_hyphen(str(i)))
df['count='] = df['domain'].apply(lambda i: count_equal(str(i)))
#Length of URL
df['url_length'] = df['domain'].apply(lambda i: url_length(str(i)))
#Hostname Length
df['hostname_length'] = df['domain'].apply(lambda i: hostname_length(str(i)))
df['sus_url'] = df['domain'].apply(lambda i: suspicious_words(str(i)))
df['count-digits']= df['domain'].apply(lambda i: digit_count(str(i)))
df['count-letters']= df['domain'].apply(lambda i: letter_count(str(i)))
df['fd_length'] = df['domain'].apply(lambda i: fd_length(str(i)))
#Length of Top Level Domain
df['tld'] = df['domain'].apply(lambda i: get_tld(str(i),fail_silently=True))
df['tld_length'] = df['tld'].apply(lambda i: tld_length(i))

In [67]:
def vec_link(url):
    vec = []
    vec.append(having_ip_address(url))
    vec.append(abnormal_url(url))
    vec.append(google_index(url))
    vec.append(count_dot(url))
    vec.append(count_www(url))
    vec.append(count_atrate(url))
    vec.append(no_of_dir(url))
    vec.append(no_of_embed(url))
    vec.append(shortening_service(url))
    vec.append(count_https(url))
    vec.append(count_http(url))
    vec.append(count_per(url))
    vec.append(count_ques(url))
    vec.append(count_hyphen(url))
    vec.append(count_equal(url))
    vec.append(url_length(url))
    vec.append(hostname_length(url))
    vec.append(suspicious_words(url))
    vec.append(digit_count(url))
    vec.append(letter_count(url))
    vec.append(fd_length(url))
    aux = get_tld(url)
    vec.append(tld_length(aux))

    return vec

## Model

In [68]:
df.columns

Index(['domain', 'label', 'use_of_ip', 'abnormal_url', 'google_index',
       'count.', 'count-www', 'count@', 'count_dir', 'count_embed_domian',
       'short_url', 'count-https', 'count-http', 'count%', 'count?', 'count-',
       'count=', 'url_length', 'hostname_length', 'sus_url', 'count-digits',
       'count-letters', 'fd_length', 'tld', 'tld_length'],
      dtype='object')

In [69]:
X = df[['use_of_ip', 'abnormal_url', 'google_index',
       'count.', 'count-www', 'count@', 'count_dir', 'count_embed_domian',
       'short_url', 'count-https', 'count-http', 'count%', 'count?', 'count-',
       'count=', 'url_length', 'hostname_length', 'sus_url', 'count-digits',
       'count-letters', 'fd_length', 'tld_length']]
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [70]:
clf_link = RandomForestClassifier(n_jobs=-1)

clf_link.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1)

In [71]:
clf_link.score(X_test, y_test)

0.8945420424334046

In [72]:
# Guardar el modelo de enlaces
joblib.dump(clf_link, 'modelo_phishing_enlaces.pkl')
print("Modelo basado en enlaces guardado exitosamente.")


Modelo basado en enlaces guardado exitosamente.


# APP

In [48]:
def extract_text_and_link(email_content):
    # Find all links in the text (http, https)
    links = re.findall(r'(https?://\S+)', email_content)
    # Remove the links from the content to leave only the text
    text = re.sub(r'https?://\S+', '', email_content).strip()
    return text, links

# Example usage
email_content = """
Hello,

This is a test email. You can find more information at the following link:

Thank you.
"""
text, links = extract_text_and_link(email_content)


In [49]:
links_vec = [vec_link(i) for i in links]
text_vec = vec_text(text, vectorizer)

In [50]:
probabilidades_text = clf_text.predict_proba(text_vec)

if len(links_vec) != 0:
    probabilidades_link = clf_link.predict_proba(links_vec)
    a = (probabilidades_link[0] + probabilidades_text) / 2
else :
    a = probabilidades_text

if a.argmax() == 1:
    print('Pishing')
else:
    print('Normal')

Normal


In [74]:
# Guardar el vectorizador
joblib.dump(vectorizer, 'vectorizer.pkl')


Modelo de texto cargado exitosamente.
Modelo de enlaces cargado exitosamente.


['vectorizer.pkl']